In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ASTRA_DB_API_KEY = os.getenv('ASTRA_DB_API_KEY')
ASTRA_DB_ENDPOINT = os.getenv('ASTRA_DB_ENDPOINT')
ASTRA_DB_KEYSPACE = os.getenv('ASTRA_DB_KEYSPACE')

In [2]:
from langchain.chains import TransformChain
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain import globals
from langchain_core.runnables import chain
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser


def load_image(inputs: dict) -> dict:
    """Load image from file and encode it as base64."""
    image_path = inputs["image_path"]
  
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    image_base64 = encode_image(image_path)
    return {"image": image_base64}

load_image_chain = TransformChain(
    input_variables=["image_path"],
    output_variables=["image"],
    transform=load_image
)


class ImageInformation(BaseModel):
    """Information about an image."""
    image_description: str = Field(description="a short description of the image")
    people_count: int = Field(description="number of humans on the picture")
    main_objects: list[str] = Field(description="list of the main objects on the picture")
    humans: int = Field(description="number of humans on the picture")
    
     
# Set verbose
globals.set_debug(True)




@chain
def image_agent(inputs: dict) -> str | list[str] | dict:
 """Invoke model with image and prompt."""
 model = ChatOpenAI(temperature=0.5, model="gpt-4o-mini", max_tokens=1024)
 msg = model.invoke(
             [HumanMessage(
             content=[
             {"type": "text", "text": inputs["prompt"]},
             {"type": "text", "text": parser.get_format_instructions()},
             {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{inputs['image']}"}},
             ])]
             )
 return msg.content



parser = JsonOutputParser(pydantic_object=ImageInformation)

def get_image_informations(image_path: str) -> dict:
   vision_prompt = """
   Given the image, provide the following information:
   - A count of how many people are in the image
   - A list of the main objects present in the image
   - A description of the image
   """
   vision_chain = load_image_chain | image_agent | parser
   return vision_chain.invoke({'image_path': f'{image_path}', 
                               'prompt': vision_prompt})

In [3]:
llm = ChatOpenAI(temperature=0.5, model="gpt-4o-mini", max_tokens=1024)

    

In [4]:
from pydantic import BaseModel, Field

class ImageInformation(BaseModel):
    """Information about an image."""
    image_description: str = Field(description="a short description of the image")
    people_count: int = Field(description="number of humans on the picture")
    main_objects: list[str] = Field(description="list of the main objects on the picture")
    humans: int = Field(description="number of humans on the picture")



structured_image_agent = llm.with_structured_output(ImageInformation)

In [5]:
result = get_image_informations("path/to/your/image.jpg")
print(result)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "image_path": "path/to/your/image.jpg",
  "prompt": "\n   Given the image, provide the following information:\n   - A count of how many people are in the image\n   - A list of the main objects present in the image\n   - A description of the image\n   "
}
[chain/start] [chain:RunnableSequence > chain:TransformChain] Entering Chain run with input:
{
  "image_path": "path/to/your/image.jpg",
  "prompt": "\n   Given the image, provide the following information:\n   - A count of how many people are in the image\n   - A list of the main objects present in the image\n   - A description of the image\n   "
}
[chain/error] [chain:RunnableSequence > chain:TransformChain] s] Chain run errored with error:
"FileNotFoundError(2, 'No such file or directory')Traceback (most recent call last):\n\n\n  File \"/home/teodorrk/projects/cogitoxinfor-agents/.myvenv/lib/python3.11/site-packages/langchain/chains/base.py\", line 160, in inv

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/image.jpg'